<p style="font-family: Arial; font-size:3.75em; color:purple; font-style:bold">
<br>FANAL ANALYSIS</p><br>

This notebook illustrates the analysys phase made in FANALIC starting from the pseudo-reconstructed data performed also in FANALIC and dealing with the topology and the ROI stuff.
Following the "FANAL" way this is done in different steps:
> * Tracks building and filtering
> * Blobs  building and filtering
> * ROI filtering

# TOPOLOGY & ROI ANALYSIS

In this section we will go through the following steps:
* Discard voxels whose energies are below the threshold, and assign their energies to closest neighbours
* Make tracks
* Order tracks according to their energies, and filter those with energy lower than threshold
* Apply the tracks filter
* Get 'hottest' track blobs
* Apply blobs filter
* Apply ROI filter

In [ ]:
%matplotlib inline
%load_ext memory_profiler
%load_ext line_profiler

%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

#### Importing general stuff

In [ ]:
import os
import sys
import numpy  as np
import tables as tb
import pandas as pd
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm

# Specific IC stuff
import invisible_cities.core.system_of_units  as units
from invisible_cities.cities.components       import city
from invisible_cities.core.configure          import configure
from invisible_cities.evm.event_model         import Voxel
from invisible_cities.reco.tbl_functions      import filters as tbl_filters
from invisible_cities.reco.paolina_functions  import make_track_graphs
from invisible_cities.reco.paolina_functions  import blob_energies

# Specific fanalIC stuff
from fanal.reco.reco_io_functions import get_reco_group_name
from fanal.ana.ana_io_functions   import get_ana_group_name

from fanal.ana.ana_io_functions   import get_event_ana_data
from fanal.ana.ana_io_functions   import get_events_ana_dict
from fanal.ana.ana_io_functions   import extend_events_ana_dict
from fanal.ana.ana_io_functions   import store_events_ana_dict
from fanal.ana.ana_io_functions   import store_events_ana_counters

from fanal.ana.ana_io_functions   import get_voxels_ana_dict
from fanal.ana.ana_io_functions   import extend_voxels_ana_dict
from fanal.ana.ana_io_functions   import store_voxels_ana_dict

from fanal.core.fanal_types  import DetName
from fanal.core.logger       import get_logger

from fanal.ana.ana_functions import get_new_energies
from fanal.ana.ana_functions import get_voxel_track_relations
from fanal.ana.ana_functions import process_tracks

#from fanal.mc.mc_utilities import print_mc_event
#from fanal.mc.mc_utilities import plot_mc_event

In [ ]:
@city
def fanal_ana(det_name,       # Detector name: 'new', 'next100', 'next500'
              event_type,     # Event type: 'bb0nu', 'Tl208', 'Bi214'
              fwhm,           # FWHM at Qbb
              voxel_size,     # Voxel size (x, y, z)
              track_Eth,      # Track energy threshold
              max_num_tracks, # Maximum number of tracks
              blob_radius,    # Blob radius
              blob_Eth,       # Blob energy threshold
              roi_Emin,       # ROI minimum energy
              roi_Emax,       # ROI maximum energy
              files_in,       # Input files
              event_range,    # Range of events to analyze: all, ... ??
              file_out,       # Output file
              compression,    # Compression: 'ZLIB1', 'ZLIB4', 'ZLIB5', 'ZLIB9', 'BLOSC5', 'BLZ4HC5'
              verbosity_level):

    ### LOGGER
    logger = get_logger('FanalAna', verbosity_level)

    ### DETECTOR NAME
    det_name = getattr(DetName, det_name)

    
    ### PRINTING GENERAL INFO
    print('\n***********************************************************************************')
    print('***** Detector: {}'.format(det_name.name))
    print('***** Analizing {} events'.format(event_type))
    print('***** Energy Resolution: {:.2f}% FWFM at Qbb'.format(fwhm / units.perCent))
    print('***** Voxel Size: ({}, {}, {}) mm'.format(voxel_size[0] / units.mm,
                                                     voxel_size[1] / units.mm,
                                                     voxel_size[2] / units.mm))
    print('***********************************************************************************\n')

    print('* Track Eth: {:4.1f} keV   Max Num Tracks: {}\n' \
          .format(track_Eth/units.keV, max_num_tracks))
    print('* Blob radius: {:.1f} mm   Blob Eth: {:4.1f} keV\n' \
          .format(blob_radius, blob_Eth / units.keV))
    print('* ROI limits: [{:4.1f}, {:4.1f}] keV\n' \
          .format(roi_Emin/units.keV, roi_Emax/units.keV))
    
    
    ### INPUT RECONSTRUCTION FILES AND GROUP
    reco_group_name = get_reco_group_name(fwhm/units.perCent, voxel_size)
    print('* {} {} input reco file names:'.format(len(files_in), event_type))
    for iFileName in files_in: print(' ', iFileName)
    print('  Reco group name: {}\n'.format(reco_group_name))    
    
    
    ### OUTPUT FILE, ITS GROUPS & ATTRIBUTES
    # Output analysis file
    oFile = tb.open_file(file_out, 'w', filters=tbl_filters(compression))

    # Analysis group Name
    ana_group_name = get_ana_group_name(fwhm/units.perCent, voxel_size)
    oFile.create_group('/', 'FANAL')
    oFile.create_group('/FANAL', ana_group_name[7:])
    
    print('* Output analysis file name:', file_out)
    print('  Ana group name: {}\n'.format(ana_group_name))

    # Attributes
    oFile.set_node_attr(ana_group_name, 'input_reco_files', files_in)
    oFile.set_node_attr(ana_group_name, 'input_reco_group', reco_group_name)
    oFile.set_node_attr(ana_group_name, 'event_type', event_type)
    oFile.set_node_attr(ana_group_name, 'energy_resolution', fwhm/units.perCent)
    oFile.set_node_attr(ana_group_name, 'track_Eth', track_Eth)
    oFile.set_node_attr(ana_group_name, 'max_num_tracks', max_num_tracks)
    oFile.set_node_attr(ana_group_name, 'blob_radius', blob_radius)
    oFile.set_node_attr(ana_group_name, 'blob_Eth', blob_Eth)
    oFile.set_node_attr(ana_group_name, 'roi_Emin', roi_Emin)
    oFile.set_node_attr(ana_group_name, 'roi_Emax', roi_Emax)    
    

    ### DATA TO STORE
    # Event counters
    simulated_events     = 0
    stored_events        = 0
    smE_filter_events    = 0
    fid_filter_events    = 0
    tracks_filter_events = 0
    blobs_filter_events  = 0
    roi_filter_events    = 0

    analyzed_events = 0
    toUpdate_events = 1

    # Dictionaries for events & voxels data
    events_dict = get_events_ana_dict()
    voxels_dict = get_voxels_ana_dict()

    events_reco_df = pd.DataFrame()
    voxels_reco_df = pd.DataFrame()
    
    ### ANALYSIS PROCEDURE
    print('* Analyzing events ...\n')

    # Looping through all the input files
    for iFileName in files_in:
        
        # Updating reconstruction counters
        with tb.open_file(iFileName, mode='r') as reco_file:
            simulated_events  += reco_file.get_node_attr(reco_group_name, 'simulated_events')
            stored_events     += reco_file.get_node_attr(reco_group_name, 'stored_events')
            smE_filter_events += reco_file.get_node_attr(reco_group_name, 'smE_filter_events')
            fid_filter_events += reco_file.get_node_attr(reco_group_name, 'fid_filter_events')
        
        # Getting the events & voxels data from the reconstruction phase
        file_events = pd.read_hdf(iFileName, reco_group_name + '/events')
        file_voxels = pd.read_hdf(iFileName, reco_group_name + '/voxels')
        
        # Updating reconstruction dataframes
        events_reco_df = pd.concat([events_reco_df, file_events], axis=0)
        voxels_reco_df = pd.concat([voxels_reco_df, file_voxels], axis=0)
        
        print('* Processing {} ...'.format(iFileName))

        
        ### Looping through all the events that passed the fiducial filter
        for event_number, event_df in file_events[file_events.fid_filter].iterrows():        

            # Updating counter of analyzed events
            analyzed_events += 1
            logger.info('Analyzing event Id: {0} ...'.format(event_number))

            # Getting event data
            event_data = get_event_ana_data()
            event_data['event_id'] = event_number
            
            event_voxels = file_voxels.loc[event_number]
            num_event_voxels = len(event_voxels)
            num_event_voxels_negli = len(event_voxels[event_voxels.negli])
            voxel_dimensions = (event_df.voxel_sizeX,
                                event_df.voxel_sizeY,
                                event_df.voxel_sizeZ)

            logger.info('  Total Voxels: {}   Negli. Voxels: {}   Voxels Size: ({:3.1f}, {:3.1f}, {:3.1f}) mm'\
                        .format(num_event_voxels, num_event_voxels_negli, voxel_dimensions[0],
                                voxel_dimensions[1], voxel_dimensions[2]))

            # If there is any negligible Voxel, distribute its energy between its neighbours,
            # if not, all voxels maintain their previous energies
            if num_event_voxels_negli:
                event_voxels_newE = get_new_energies(event_voxels)
            else:
                event_voxels_newE = event_voxels.E.tolist()
        
            # Translate fanalIC voxels info to IC voxels to make tracks
            #ic_voxels = [Voxel(event_voxels.iloc[i].X, event_voxels.iloc[i].Y, event_voxels.iloc[i].Z,
            #                   event_voxels_newE[i], voxel_dimensions) for i in range(num_event_voxels)]
            ic_voxels = []
            for i, voxel in event_voxels.iterrows():
                ic_voxel = Voxel(voxel.X, voxel.Y, voxel.Z, event_voxels_newE[i], voxel_dimensions)
                ic_voxels.append(ic_voxel)            
            
            # Make tracks
            event_tracks = make_track_graphs(ic_voxels)
            num_ini_tracks = len(event_tracks)
            logger.info('  Num initial tracks: {:2}'.format(num_ini_tracks))

            # Appending to every voxel, the track it belongs to
            event_voxels_tracks = get_voxel_track_relations(event_voxels, event_tracks)

            # Appending ana-info to this event voxels
            extend_voxels_ana_dict(voxels_dict, event_number, event_voxels.index.tolist(),
                                   event_voxels_newE, event_voxels_tracks)

            # Processing tracks: Getting energies, sorting and filtering ...
            event_sorted_tracks = process_tracks(event_tracks, track_Eth)         
            event_data['num_tracks'] = len(event_sorted_tracks)

            # Getting 3 hottest tracks info
            if event_data['num_tracks'] >= 1:
                event_data['track0_E']      = event_sorted_tracks[0][0]
                event_data['track0_length'] = event_sorted_tracks[0][1]
                event_data['track0_voxels'] = len(event_sorted_tracks[0][2].nodes())
            if event_data['num_tracks'] >= 2:
                event_data['track1_E']      = event_sorted_tracks[1][0]
                event_data['track1_length'] = event_sorted_tracks[1][1]
                event_data['track1_voxels'] = len(event_sorted_tracks[1][2].nodes())
            if event_data['num_tracks'] >= 3:
                event_data['track2_E']      = event_sorted_tracks[2][0]
                event_data['track2_length'] = event_sorted_tracks[2][1]
                event_data['track2_voxels'] = len(event_sorted_tracks[2][2].nodes())
            
            # Applying the tracks filter consisting on:
            # 0 < num tracks < max_num_tracks
            # the track length must be longer than 2 times the blob_radius
            event_data['tracks_filter'] = ((event_data['num_tracks'] >  0) &
                                           (event_data['num_tracks'] <= max_num_tracks) &
                                           (event_data['track0_length'] >=  2. * blob_radius))
        
            # Verbosing
            logger.info('  Num final tracks: {:2}  -->  tracks_filter: {}' \
                        .format(event_data['num_tracks'], event_data['tracks_filter']))

            
            ### For those events passing the tracks filter:
            if event_data['tracks_filter']:

                # Getting the blob energies of the track with highest energy
                event_data['blob1_E'], event_data['blob2_E'] = \
                    blob_energies(event_sorted_tracks[0][2], blob_radius)
                
                # Applying the blobs filter
                event_data['blobs_filter'] = (event_data['blob2_E'] > blob_Eth)
               
                # Verbosing
                logger.info('  Blob 1 energy: {:4.1f} keV   Blob 2 energy: {:4.1f} keV  -->  Blobs filter: {}'\
                            .format(event_data['blob1_E']/units.keV, event_data['blob2_E']/units.keV,
                                    event_data['blobs_filter']))

                
                ### For those events passing the blobs filter:
                if event_data['blobs_filter']:

                    # Getting the total event smeared energy
                    event_smE = file_events.loc[event_number].smE
                    
                    # Applying the ROI filter
                    event_data['roi_filter'] = ((event_smE >= roi_Emin) & (event_smE <= roi_Emax))
                
                    # Verbosing
                    logger.info('  Event energy: {:6.1f} keV  -->  ROI filter: {}'\
                                .format(event_smE / units.keV, event_data['roi_filter']))  

                    
            # Storing event_data
            extend_events_ana_dict(events_dict, event_data)

            # Verbosing
            if (not(analyzed_events % toUpdate_events)):
                print('* Num analyzed events: {}'.format(analyzed_events))
            if (analyzed_events == (10 * toUpdate_events)): toUpdate_events *= 10
    
    
    ### STORING ANALYSIS DATA
    print('* Total analyzed events: {}'.format(analyzed_events))

    # Storing events and voxels dataframes
    print('\n* Storing data in the output file ...\n  {}\n'.format(file_out))
    store_events_ana_dict(file_out, ana_group_name, events_reco_df, events_dict)
    store_voxels_ana_dict(file_out, ana_group_name, voxels_reco_df, voxels_dict)

    # Storing event counters as attributes
    tracks_filter_events = sum(events_dict['tracks_filter'])
    blobs_filter_events  = sum(events_dict['blobs_filter'])
    roi_filter_events    = sum(events_dict['roi_filter'])
    
    store_events_ana_counters(oFile, ana_group_name,
                              simulated_events, stored_events,
                              smE_filter_events, fid_filter_events,
                              tracks_filter_events, blobs_filter_events,
                              roi_filter_events)
    
    
    ### Ending ...
    oFile.close()
    print('* Analysis done !!\n')
    
    print('''* Event counters:
  Simulated events:     {0:9}
  Stored events:        {1:9}
  smE_filter events:    {2:9}
  fid_filter events:    {3:9}
  tracks_filter events: {4:9}
  blobs_filter events:  {5:9}
  roi_filter events:    {6:9}'''
          .format(simulated_events, stored_events, smE_filter_events,
                  fid_filter_events, tracks_filter_events, blobs_filter_events,
                  roi_filter_events))


In [ ]:
from ana_sets import analysis
print(analysis.keys())

# RUNNING THE ANALYSIS

In [ ]:
%%time

current_ana     = analysis['next100_bb0nu_07_10x10x10']
compression     = 'ZLIB4'
verbosity_level = 'WARNING'

fanal_ana(det_name        = current_ana['det_name'],
          event_type      = current_ana['event_type'],
          fwhm            = current_ana['fwhm'],
          voxel_size      = current_ana['voxel_size'],
          track_Eth       = current_ana['track_Eth'],
          max_num_tracks  = current_ana['max_num_tracks'],
          blob_radius     = current_ana['blob_radius'],
          blob_Eth        = current_ana['blob_Eth'],
          roi_Emin        = current_ana['roi_Emin'],
          roi_Emax        = current_ana['roi_Emax'],
          files_in        = current_ana['files_in'],
          event_range     = current_ana['event_range'],
          file_out        = current_ana['file_out'],
          compression     = compression,
          verbosity_level = verbosity_level)


# VERBOSING THE ANALYSIS ...

In [ ]:
DET_NAME        = getattr(DetName, current_ana['det_name'])

RECO_GROUP_NAME = get_reco_group_name(current_ana['fwhm'] / units.perCent, current_ana['voxel_size'])

ANA_FILE_NAME   = current_ana['file_out']
ANA_GROUP_NAME  = get_ana_group_name(current_ana['fwhm'] / units.perCent, current_ana['voxel_size'])

#### Verbosing the event counters

In [ ]:
with tb.open_file(ANA_FILE_NAME, mode='r') as iFile:
    simulated_events     = iFile.get_node_attr(ANA_GROUP_NAME, 'simulated_events')
    stored_events        = iFile.get_node_attr(ANA_GROUP_NAME, 'stored_events')
    smE_filter_events    = iFile.get_node_attr(ANA_GROUP_NAME, 'smE_filter_events')
    fid_filter_events    = iFile.get_node_attr(ANA_GROUP_NAME, 'fid_filter_events')
    tracks_filter_events = iFile.get_node_attr(ANA_GROUP_NAME, 'tracks_filter_events')
    blobs_filter_events  = iFile.get_node_attr(ANA_GROUP_NAME, 'blobs_filter_events')
    roi_filter_events    = iFile.get_node_attr(ANA_GROUP_NAME, 'roi_filter_events')


print('''* Event counters:
  Simulated events:     {0:8}
  Stored events:        {1:8}
  smE_filter events:    {2:8}
  fid_filter events:    {3:8}
  tracks_filter events: {4:8}
  blobs_filter events:  {5:8}
  roi_filter events:    {6:8}'''
      .format(simulated_events, stored_events, smE_filter_events,
              fid_filter_events, tracks_filter_events,
              blobs_filter_events, roi_filter_events))


#### Verbosing the dataframes

In [ ]:
events_df = pd.read_hdf(ANA_FILE_NAME, ANA_GROUP_NAME + '/events')
voxels_df = pd.read_hdf(ANA_FILE_NAME, ANA_GROUP_NAME + '/voxels')

In [ ]:
events_df.head(12)

In [ ]:
voxels_df.head(10)

In [ ]:
events_df_smE_True    = events_df[events_df.smE_filter]
events_df_fid_True    = events_df[events_df.fid_filter]
events_df_tracks_True = events_df[events_df.tracks_filter]
events_df_blobs_True  = events_df[events_df.blobs_filter]
events_df_roi_True    = events_df[events_df.roi_filter]

In [ ]:
columns_toShow = ['smE', 'num_tracks', 'track0_E', 'track0_voxels', 'track0_length',
                  'tracks_filter', 'blob1_E', 'blob2_E', 'blobs_filter', 'roi_filter']
#events_df_fid_True[columns_toShow].head()
events_df_tracks_True[columns_toShow].head()
#events_df_blobs_True[columns_toShow].head()
#events_df_roi_True[columns_toShow].head()

In [ ]:
#events_df_tracks_True.columns
columns_toShow = ['num_MChits', 'smE', 'smE_filter', 'num_voxels', 'voxel_sizeX', 'voxel_sizeY',
                  'voxel_sizeZ', 'voxels_minZ', 'voxels_maxZ', 'voxels_maxRad',
                  'veto_energy', 'fid_filter', 'num_tracks', 'blob1_E', 'blob2_E']
events_df_fid_True[events_df_fid_True.num_voxels <= 10][columns_toShow].head()

In [ ]:
#voxels_df.loc[214].head()

#### Generating some histograms ...

In [ ]:
fig = plt.figure(figsize = (8,4))

counters_toPlot = {'smE_filter Events': smE_filter_events,
                   'fid_filter Events': fid_filter_events,
                   'tracks_filter Events': tracks_filter_events,
                   'blobs_filter Events': blobs_filter_events,
                   'roi_filter Events': roi_filter_events}

plt.bar(range(len(counters_toPlot)), list(counters_toPlot.values()), align='center',
        color=['b', 'r', 'c', 'y', 'm'], yerr=np.sqrt([smE_filter_events, fid_filter_events,
                                                       tracks_filter_events, blobs_filter_events,
                                                       roi_filter_events]))
plt.xticks(range(len(counters_toPlot)), list(counters_toPlot.keys()), rotation='30')
plt.show()

Plotting number of tracks

In [ ]:
fig = plt.figure(figsize = (8,5))
num_bins = 10

#plt.hist(events_df_fid_True.num_tracks, num_bins, [0, 10])
plt.hist(events_df_fid_True.num_tracks, num_bins, [0, 10])
plt.xlabel('Num Tracks', size=12)
plt.ylabel('Num. events', size=12)
plt.title('{} - Number of reconstructed tracks'.format(current_ana['event_type']))

Plotting energies, number of voxels and lengths of the three hottest tracks

In [ ]:
fig = plt.figure(figsize = (18, 18))
num_E_bins = 20
num_voxels_bins = 20
num_length_bins = 20

# First track plots
plotting_events = events_df_fid_True[events_df_fid_True.num_tracks >= 1]
ax1 = fig.add_subplot(3, 3, 1)
plt.hist(plotting_events.track0_E, num_E_bins, [0., 2.5])
plt.title('Track 0 Energy [MeV]', size=12)

ax2 = fig.add_subplot(3, 3, 4)
plt.hist(plotting_events.track0_voxels, num_voxels_bins, [0, 40])
plt.title('Track 0 - Num Voxels', size=12)

ax3 = fig.add_subplot(3, 3, 7)
plt.hist(plotting_events.track0_length, num_length_bins, [0, 200])
plt.title('Track 0 - Length', size=12)

# Second track plots
plotting_events = events_df_fid_True[events_df_fid_True.num_tracks >= 2]
ax4 = fig.add_subplot(3, 3, 2)
plt.hist(plotting_events.track1_E, num_E_bins, [0., 1.0])
plt.title('Track 1 Energy [MeV]', size=12)

ax5 = fig.add_subplot(3, 3, 5)
plt.hist(plotting_events.track1_voxels, num_voxels_bins, [0, 20])
plt.title('Track 1 - Num Voxels', size=12)

ax6 = fig.add_subplot(3, 3, 8)
plt.hist(plotting_events.track1_length, num_length_bins, [0, 100])
plt.title('Track 1 - Length', size=12)

# Third track plots
plotting_events = events_df_fid_True[events_df_fid_True.num_tracks >= 3]
ax7 = fig.add_subplot(3, 3, 3)
plt.hist(plotting_events.track2_E, num_E_bins, [0., 1.0])
plt.title('Track 2 Energy [MeV]', size=12)

ax8 = fig.add_subplot(3, 3, 6)
plt.hist(plotting_events.track2_voxels, num_voxels_bins, [0, 20])
plt.title('Track 2 - Num Voxels', size=12)

ax9 = fig.add_subplot(3, 3, 9)
plt.hist(plotting_events.track2_length, num_length_bins, [0, 50])
plt.title('Track 2 - Length', size=12)


Energy histogram of events passing the tracks filter 

In [ ]:
fig = plt.figure(figsize = (15,5))
num_bins = 20

ax1 = fig.add_subplot(1, 2, 1)
plt.hist(events_df_tracks_True.track0_E, num_bins, [2.4, 2.5])
plt.xlabel('Track Energy [MeV]', size=14)
plt.title('{} - Track 0 Energy'.format(current_ana['event_type']))

ax2 = fig.add_subplot(1, 2, 2)
plt.hist(events_df_tracks_True.smE, num_bins, [2.4, 2.5])
plt.xlabel('Event Energy [MeV]', size=14)
plt.title('{} - Event Energy'.format(current_ana['event_type']))

# As expected, both histograms are the same


Histograms of blob energies ...

In [ ]:
fig = plt.figure(figsize = (15,5))
num_bins = 30

ax1 = fig.add_subplot(1, 2, 1)
plt.hist(events_df_tracks_True.blob1_E, num_bins, [0, 1.5])
plt.xlabel('Blob1 Energy [MeV]')
plt.title('{} - Blob1 Energy [MeV]'.format(current_ana['event_type']), size=14)

ax2 = fig.add_subplot(1, 2, 2)
plt.hist(events_df_tracks_True.blob2_E, num_bins, [0, 1.5])
plt.xlabel('Blob2 Energy [MeV]')
plt.title('{} - Blob2 Energy [MeV]'.format(current_ana['event_type']), size=14)


fig = plt.figure(figsize = (7,6))
plt.hist2d(events_df_tracks_True.blob1_E, events_df_tracks_True.blob2_E, num_bins,
           [[0, 1.5], [0, 1.5]], norm=LogNorm())
plt.xlabel('Highest Blob Energy [MeV]')
plt.ylabel('Lowest Blob Energy [MeV]')
plt.title('{} - Blobs Energies [MeV]'.format(current_ana['event_type']), size=14)
plt.colorbar()

ROI energy ...

In [ ]:
with tb.open_file(ANA_FILE_NAME, mode='r') as iFile:
    ROI_E_MIN = iFile.get_node_attr(ANA_GROUP_NAME, 'roi_Emin')
    ROI_E_MAX  = iFile.get_node_attr(ANA_GROUP_NAME, 'roi_Emax')

fig = plt.figure(figsize = (7,6))
num_bins = int((ROI_E_MAX - ROI_E_MIN) / units.keV)

plt.hist(events_df_roi_True.smE, num_bins, [ROI_E_MIN, ROI_E_MAX])
plt.xlabel('Event Energy [MeV]')
plt.title('{} - Event Energy'.format(current_ana['event_type']), size=14)


In [ ]:
voxels_df[voxels_df.negli].head()

In [ ]:
columns_toShow = ['E', 'newE', 'track_id']
voxels_df.loc[events_df_fid_True.index, columns_toShow]

Printing and plotting the MC info of an event ...

In [ ]:
#event_to_print = 149
#
#sim_files = []
#with tb.open_file(ANA_FILE_NAME, mode='r') as iFile:
#    reco_file_names = iFile.get_node_attr(ANA_GROUP_NAME, 'input_reco_files')
#    
#    for reco_file_name in reco_file_names:
#        with tb.open_file(reco_file_name, mode='r') as iRecoFile:
#            sim_files += iRecoFile.get_node_attr(RECO_GROUP_NAME, 'input_sim_files')
#            
#print_mc_event(event_to_print, sim_files, with_hits=True)

In [ ]:
#print(files_list)
#event_to_plot = 149
#plot_mc_event(event_to_plot, sim_files)

In [ ]:
            event_voxels = file_voxels.loc[event_number]
            voxel_dimensions = (event_df.voxel_sizeX,
                                event_df.voxel_sizeY,
                                event_df.voxel_sizeZ)

            ic_voxels = []
            for voxel in event_voxels.iterrows():
                ic_voxel = Voxel(voxel.X, voxel.Y, voxel.Z, voxel.E, voxel_dimensions)
                ic_voxels.append(ic_voxel)            
            
            # Make tracks
            event_tracks = make_track_graphs(ic_voxels)
